# Chapter 8 - Understanding Functions

## FP = Functions, Functions, Everywhere
* Programs are made **exclusively** by applying and composing functions

## First Class Functions
* Functions can:
  * be passed as arguments to other functions
  * be returned as values from other functions
  * be assigned to variables
  * be stored in data structures
  * be defined as anonymous functions

In [10]:
let plus3 x = x + 3
let addThree = plus3
let square = fun x -> x * x

let listOfFunctions = [plus3; addThree; square]

for fn in listOfFunctions do
  let result = fn 100
  printf "If the input is 100, the output is %i\n" result



If the input is 100, the output is 103
If the input is 100, the output is 103
If the input is 100, the output is 10000


### Functions as Inputs

In [11]:
let evalWith5ThenAdd2 fn = fn(5) + 2 // fn is inferred as (int -> int)

evalWith5ThenAdd2 plus3
evalWith5ThenAdd2 square

27

### Functions as Outputs 

In [12]:
let addN n = fun x -> n + x
let add1 = addN 1
let add2 = addN 2
let add3 = addN 3

printfn "%i" (add1 10)
printfn "%i" (add2 10)
printfn "%i" (add3 10)

11
12
13


### Currying and Partial Application
* Any multiparameter function can be converted into a series of single-parameter functions
* In F#, every function is a curried function
* You can "fix" the first argument of a function, to get a function of the remaining arguments
* Something I found interesting: [Why Roc has decided against auto-currying](https://www.roc-lang.org/faq.html#curried-functions) 

In [13]:
let add x y = x + y
let add1 = add 1

printfn "%i" (add1 10)

let sayGreeting greeting name = printfn "%s %s" greeting name
let sayHello = sayGreeting "Hello"
let sayGoodbye = sayGreeting "Goodbye"

sayHello "Victor"
sayGoodbye "Victor"

11
Hello Victor
Goodbye Victor


### Total Functions
* A total function maps each possible input to an output
  * Examples include square, exponential, polynomial, empty function
* In a partial function, one or more inputs aren't mapped to an output
  * Boring example is division as division by 0 is undefined (on the Reals)
* In codeland, total functions let us make things explicit by documenting all effects in the type signature

![partial-function.png](https://www.statisticshowto.com/wp-content/uploads/2019/12/partial-function.png)


In [14]:
// Ignoring the incomplate pattern match, the type signature (int -> int) is a lie because we get an exception sometimes 😱
let twelveDividedBy n 
  = match n with
    | 6 -> 2
    | 4 -> 3
    | 3 -> 4
    | 2 -> 6
    | 1 -> 2
    | 0 -> failwith "Can't divide by zero"

// Option 1: restrict the input 
type NonZeroInteger = private NonZeroInteger of int // this doesn't look practical 

let twelveDividedByWithRestrictedInput (NonZeroInteger n) 
  = match n with
    | 6 -> 2
    | 4 -> 3
    | 3 -> 4
    | 2 -> 6
    | 1 -> 2

// Option 2: extend the output
let twelveDividedByWithExtendedOutput n
  = match n with
    | 6 -> Some 2
    | 4 -> Some 3
    | 3 -> Some 4
    | 2 -> Some 6
    | 1 -> Some 2
    | 0 -> None

In [19]:
// The perils of lying type signatures in TS
const { open } = require('node:fs/promises');

async function readFile() {
  const handle = await open('./some/file/to/read');
  return handle.readFile(file, 'utf-8')
}
// apparently readFile: Promise<Buffer> but what if the file is missing 🤔

const parsedFile = readFile().then((buf) => buf.toString('utf8')).then(JSON.parse)
// no we're dealing with Promise<any> but JSON.parse also throws

Error: Module name "node:fs/promises" has not been loaded yet for context: _. Use require([])
https://requirejs.org/docs/errors.html#notloaded

## Composition 

* Combining functions by connecting the output of the first to the input of the second
* One benefit is **information hinding** - the caller cannot tell whether a function is composed of smaller functions, nor what they operated on
* In F#, the pipe operator `|>` can be used to compose functions as long as the output type of the first matches the input type of the second


In [17]:
let add1 x = x + 1
let square x = x * x
let add1ThenSquare x = x |> add1 |> square
printfn "%i" (add1ThenSquare 4)

let isEven x = (x % 2) = 0
let printBoolean x = sprintf "value is %b" x
let isEvenThenPrint x = x |> isEven |> printBoolean
isEvenThenPrint 2


25


value is true

```clojure
;; Our trusty thread operator
(->>
  (on-event component)
  (record->event-payload-middleware component)
  (event-markers/deduplication-middleware)
  (record->event-href-middleware component)
  (event-markers/record->event-marker-middleware)
  (component->database-context-middleware component)
  (dd-utils/span-middleware "parallel_consumer.handle_event")
  (pc/consumer-record->record-middleware)
  (pc/start configuration))
```

### Can use composition to build the entire application
* "Service" = Low-level operation 1 |> Low-level operation 2 |> Low-level operation 1
* "Workflow" = Serice A |> Service B
* "Controller/dispatcher" input -> invoke Workflow X or Workflow Y

### Challenges with composition
* Typically with wrapper types like `Option`, `Result`, `Async`
* Requiring us to "adjust" the inputs and outputs of functions so they can be composed
* Typical approach is to use the "simplest" type that encompasses both
  * e.g `functionA` outputs `int` and `functionB` takes an `Option<int>`, we can can convert the output of `functionA` to `Option<int>`



In [18]:
let add1 x = x + 1
let printOption x
  = match x with
    | Some i -> printfn "The int is %i" i
    | None -> printfn "No value"

5 |> add1 |> Some |> printOption

The int is 6


## What's Next?

Finally some implementation 🚀
